<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/BigQuery_datawarehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install kafka-python

In [27]:
from google.colab import auth
auth.authenticate_user()
print("authorized")

authorized


In [0]:
from kafka import KafkaProducer, KafkaConsumer
from kafka.errors import KafkaError
from json import dumps, loads
import time
from google.cloud import bigquery
project_id = 'e6820-235222'
client = bigquery.Client(project=project_id)

In [0]:
consumer = KafkaConsumer(
    'tweepyv1',
     bootstrap_servers=['35.243.144.79:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='my-gu333',
     value_deserializer=lambda x: loads(x.decode('utf-8')))
def make_unicode(input):
    if type(input) != unicode:
        input =  input.decode('utf-8')
        return input
    else:
        return input

In [68]:
%%bigquery --project e6820-235222 df
create table warehouse.tweet_full6(
created_at timestamp,
entities_hastages string,
favorite_count int64,
id int64,
id_str string,
lang string,
place_country string,
place_country_code string,
place_name string,
quote_count int64,
quoted_status_entities_hashtages string,
quoted_status_extended_tweet_entites_hashtags string,
quoted_status_extended_tweet_full_text string,
quoted_status_favorite_count int64,
quoted_status_id_str string,
reply_count int64,
retweet_count int64,
text string,
timestamp_ms string,
user_created_at string,
user_description string,
user_favourites_count int64,
user_followers_count int64,
user_id int64,
user_id_str string,
user_listed_count int64,
user_location string,
user_name string,
latitude string,
longitude string
)

""


In [0]:
def switch_month(argument):
    switcher = {
        "Jan":'1',
        "Feb":'2',
        "Mar":'3',
        "Apr":'4',
        "May":'5',
        "Jun":'6',
        "Jul":'7',
        "Aug":'8',
        "Sep":'9',
        "Oct":'10',
        "Nov":'11',
        "Dec":'12'
    }
    return switcher.get(argument, "Invalid month")

In [0]:
for line in consumer:
    
  entities_hastages = []
  for a in res[-6]['entities']['hashtags']:
    entities_hastages.append(a['text'])
#   if entities_hastages == []:
  entities_hastages = str(entities_hastages)
  
  res = loads(dumps(line))
  favorite_count = int(res[-6]['favorite_count'])
  id = res[-6]['id']
  quote_count= res[-6]['quote_count'] 
  reply_count = res[-6]['reply_count']
  retweet_count = res[-6]['retweet_count']
  user_favourites_count = res[-6]['user']['favourites_count'] 
  user_followers_count =  res[-6]['user']['followers_count']
  user_id =res[-6]['user']['id']
  user_listed_count =  res[-6]['user']['listed_count']
  
  created_at = str(res[-6]['created_at'])
  tmp = created_at.split(" ")
  created_at_ = tmp[5]+ "-" + switch_month(tmp[1]) + "-" + tmp[2] + " " +tmp[3]
  
  id_str = str(res[-6]['id'])
  lang = str(res[-6]['lang'])
  place_country = str(res[-6]['place']['country'])
  place_country_code = str(res[-6]['place']['country_code'])
  place_name = str(res[-6]['place']['name'])
#   quoted_status_entities_hashtages = str(res[-6]['quoted_status']['entities']['hashtages'])
#   extended_tweet_entites_hashtags = str(res[-6]['extended_tweet']['entities']['hashtags'])
#   extended_tweet_full_text = str(res[-6]['extended_tweet']['full_text'])
#   extended_tweet_id_str = str(res[-6]['extended_tweet']['id_str'])
  text = str(res[-6]['text'])
  timestamp_ms = str(res[-6]['timestamp_ms'])
  user_created_at = str(res[-6]['user']['created_at'])
  user_description = str(res[-6]['user']['description'])
  user_id_str = str(res[-6]['user']['id_str'])
  user_location = str(res[-6]['user']['location'])
  user_name = str(res[-6]['user']['name'])
  latitude = str(res[-6]['place']['bounding_box']['coordinates'][0][0][0])
  longitude = str(res[-6]['place']['bounding_box']['coordinates'][0][0][1])
#   print(created_at, entities_hastages, id_str, lang, place_country, place_country_code, place_name,  \
#           text, timestamp_ms, user_created_at, user_description, user_id_str, user_location, user_name)
#   print(favorite_count, id, quote_count, reply_count, \
#   retweet_count, user_favourites_count, user_followers_count, user_id, user_listed_count)

  query_job = "INSERT warehouse.tweet_full6 (favorite_count, id, quote_count, reply_count, \
  retweet_count, user_favourites_count, user_followers_count, user_id, user_listed_count, created_at, entities_hastages, id_str, lang, place_country, place_country_code, place_name, \
          text, timestamp_ms, user_created_at, user_description, user_id_str, user_location, user_name, latitude, longitude) \
  VALUES ({}, {}, {}, {}, {}, {}, {}, {}, {}".format(favorite_count, id, quote_count, reply_count, \
  retweet_count, user_favourites_count, user_followers_count, user_id, user_listed_count) + ", '"+ created_at_ + "', \""+ entities_hastages+ "\", '"+ id_str+ "', '"+ lang+ "', '"+ place_country+ "', '"+ place_country_code+ "', '"+ place_name+ "', '"+  \
          text+ "', '"+ timestamp_ms+ "', '"+ user_created_at+ "', '"+ user_description+ "', '"+ user_id_str+ "', '"+ user_location+ "', '"+ user_name+ "','" +latitude + "','"+longitude +"')"
#   query_job = "INSERT into warehouse.tweet_full3 (entities_hastages) VALUES ('"+a +"')"

  client.query(
      query_job,
  )

[] -88.473228 30.144425
['AlreadyBeen', 'AvengersEndgame'] -71.01249 42.694927
['GameofThrones'] -76.902529 38.825445
[] -81.395067 40.374291
[] -77.119401 38.801826
[] -97.51982 35.15327
[] -91.161353 41.35847
[] -92.734385 32.50363
[] -75.548035 40.547616
[] -79.639319 43.403221
[] -115.316075 35.960753
[] -114.316038 50.84278
[] -124.482003 32.528832
[] -118.668404 33.704538
[] -81.240943 42.743002
[] -112.323914 33.29026
[] -83.697158 42.851374
[] -124.703541 41.991795
[] -117.86894 33.606077
[] -113.71368 53.395531
[] -73.938935 43.003668
[] -112.62655 33.355798
[] -90.418108 41.696088
[] -112.323914 33.29026
[] -87.721811 34.77898
[] -122.461455 49.002189
[] -90.782468 35.763136
[] -97.993537 29.840708
[] -118.668404 33.704538
['AvergersEndgame'] -96.015096 30.061706
[] -115.384091 36.129459
[] -84.561205 33.788139
[] -87.022482 35.994568
['AryaStark', 'GameofThronesSpoilers', 'GameofThrones'] -90.185267 38.686586
[] -87.940033 41.644102
[] -95.823268 29.522325
[] -95.774704 35.9

In [0]:
import uuid
def stream_data(self, table, data, schema):
    # first checks if table already exists. If it doesn't, then create it
    r = self.service.tables().list(projectId=your_project_id,
                                     datasetId=your_dataset_id).execute()
    table_exists = [row['tableReference']['tableId'] for row in
                    r['tables'] if
                    row['tableReference']['tableId'] == table]
    if not table_exists:
        body = {
            'tableReference': {
                'tableId': table,
                'projectId': your_project_id,
                'datasetId': your_dataset_id
            },
            'schema': schema
        }
        self.service.tables().insert(projectId=your_project_id,
                                     datasetId=your_dataset_id,
                                     body=body).execute()

    # with table created, now we can stream the data
    # to do so we'll use the tabledata().insertall() function.
    body = {
        'rows': [
            {
                'json': data,
                'insertId': str(uuid.uuid4())
            }
        ]
    }
    self.service.tabledata().insertAll(projectId=your_project_id),
                                       datasetId=your_dataset_id,
                                       tableId=table,
                                         body=body).execute(num_retries=5)

In [0]:
message = consumer.poll(timeout_ms=10000, max_records=1)
for key in message.keys():
  if type(key) is not str:
    try:
      message[str(key)] = message[key]
    except:
      try:
        message[repr(key)] =messagemydict[key]
      except:
        pass
    del message[key]
    
a =loads(dumps(message))["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]
# print(type(loads(dumps(message))["TopicPartition(topic='tweepyv1', partition=0)"][-1][-6]))

In [19]:
# client.insert_rows("`warehouse.tweet`", str(string), [('tweets','STRING')])

# print(loads(dumps(message)))
query_params = [bigquery.ScalarQueryParameter('tweets', 'STRING', 'message{}'.format(1))]
queryjob = "INSERT warehouse.tweet_info (full_text) VALUES (\""+a +"\")"
print(queryjob)
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
client.query(
    queryjob,
    job_config=job_config
)


INSERT warehouse.tweet_info (full_text) VALUES ("['GameOfThornes', 'nottoday']")


In [0]:
for message in consumer:
    key, value = message.key, message.value
#     value.decode()
    print(key)

In [15]:
a = "['GameOfThornes', 'nottoday']"
print(type(a))

<class 'str'>


In [66]:
a = 'Mon Apr 29 04:45:26 +0000 2019'
a = a.split(" ")
timest = a[5]+ "-" + switch_month(a[1]) + "-" + a[2] + " " +a[3]

2019-4-29 04:45:26


'1'